In [12]:
import re
import os.path
from bs4 import BeautifulSoup, BeautifulStoneSoup

def convert(file: str) -> dict:
    questions = re.findall(r'"textarea_question_text">([\s\S]*?)<', file)
    answers = re.findall(
        r'"(.*?)(?:\.*?)(?:. This was the correct answer|. You selected this answer. This was the correct answer.)', file)

    raw_pairs = dict(zip(questions, answers))
    pairs = {}

    for pair in raw_pairs:
        if '&lt' not in pair and 'All of the above' not in raw_pairs[pair]:
            pairs[pair] = raw_pairs[pair]

    pairs = {k: v for k, v in pairs.items() if len(v) >= 1}
    return pairs

In [13]:
def write_pairs(pairs: dict, location: str):
    with open(location, 'w', encoding="utf8") as f:
        for key in pairs.keys():
            f.write(f"{key}\t{pairs[key]}\n")

def perform_actions(input_path):
    split = os.path.split(input_path)
    output_path = split[0] + "/" + split[1].split('.')[0] + "-output.txt"

    file = open(input_path, "r", encoding="utf8").read().strip()

    html_decoded_string = BeautifulSoup(file, "lxml")

    pairs = convert(html_decoded_string)
    write_pairs(pairs, output_path)

    print(f'Wrote to {output_path}.')

def get_HTML_paths_from_directory(dir_path):
    results = []
    for file in os.listdir(dir_path):
        if file.endswith(".html"):
            results.append(os.path.join(dir_path, file))
    return results


In [14]:
def main():
    # input_path = input("Enter the Input File Path")
    input_path = "/Users/zak/dev/canvas-to-quizlet/examples/frame1.html"
    perform_actions(input_path)

    html_paths = get_HTML_paths_from_directory("/Users/zak/dev/canvas-to-quizlet/examples")
    
    # for f in html_paths:
        # perform_actions(f)
    


In [15]:
if __name__ == "__main__":
    main()

<!DOCTYPE html>
<html class="scripts-not-loaded" dir="ltr"   lang="en">
<head>
  <meta charset="utf-8">
  <meta name="sentry-trace" content="4b7280808a36401592a47ebe620d18bc-5592e2b47f200f65-0"/>
  <link rel="preload" href="https://du11hjcvx0uqb.cloudfront.net/dist/fonts/lato/extended/Lato-Regular-bd03a2cc27.woff2" as="font" type="font/woff2" crossorigin="anonmyous">
  <link rel="preload" href="https://du11hjcvx0uqb.cloudfront.net/dist/fonts/lato/extended/Lato-Bold-cccb897485.woff2" as="font" type="font/woff2" crossorigin="anonmyous">
  <link rel="preload" href="https://du11hjcvx0uqb.cloudfront.net/dist/fonts/lato/extended/Lato-Italic-4eb103b4d1.woff2" as="font" type="font/woff2" crossorigin="anonmyous">
  <link rel="stylesheet" media="screen" href="https://du11hjcvx0uqb.cloudfront.net/dist/brandable_css/no_variables/bundles/fonts-43e9c545fc.css" />
    <script>if (navigator.userAgent.match(/(MSIE|Trident\/)/)) location.replace('/ie-is-not-supported.html')</script>
  
  <link rel="shor

TypeError: expected string or bytes-like object